In [1]:
# ============================================================================
# FPL Model Tuning Notebook
# ============================================================================
# This notebook demonstrates the tune mode for evaluating and optimizing models.
# Use this to:
#   - Evaluate model performance with proper train/test splits
#   - Compare different hyperparameter configurations
#   - Run feature selection (importance-based or RFE)
#   - Run Optuna hyperparameter optimization
#   - View feature importance and model metrics
# ============================================================================

import pandas as pd
import numpy as np
import warnings
from pathlib import Path
import os

warnings.filterwarnings('ignore')

# Set up path - only change if not already in project root
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir("..")
print(f"Working directory: {os.getcwd()}")

# Import the pipeline
from pipelines.fpl_prediction_pipeline import FPLPredictionPipeline, TuneConfig, ModelMetrics


Working directory: c:\Users\dpfin\OneDrive\Desktop\ultimate_p_proj


In [2]:
#from update_data import get_current_and_next_gw, get_next_gw_fixtures, get_players_for_teams
from pipelines.fpl_prediction_pipeline import FPLPredictionPipeline, TuneConfig, load_raw_data, compute_rolling_features

In [3]:
# ============================================================================
# GW17 PREDICTIONS - Using Tuned Models
# ============================================================================
# This cell:
# 1. Scrapes GW16 data from FBRef (if not already present)
# 2. Gets GW17 fixtures from FPL API
# 3. Runs predictions using the best tuned model configurations
# 4. Prints model accuracy summaries and player projections table
# ============================================================================

import sys
import requests
import json
from pathlib import Path
from datetime import datetime

# Add scripts to path
sys.path.insert(0, str(Path('scripts').resolve()))

from update_data import get_current_and_next_gw, get_next_gw_fixtures, get_players_for_teams
from pipelines.fpl_prediction_pipeline import FPLPredictionPipeline, TuneConfig

DATA_DIR = Path(r'c:\Users\dpfin\OneDrive\Desktop\ultimate_p_proj\data')
SEASON = "2025-26"

print("=" * 80)
print("GW17 2025-26 PREDICTIONS USING TUNED MODELS")
print("=" * 80)
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# ============================================================================
# STEP 1: Check current gameweek status from FPL API
# ============================================================================
print("STEP 1: Checking FPL API for gameweek status...")
current_gw, next_gw = get_current_and_next_gw()
print(f"  Current GW: {current_gw} (just completed/in progress)")
print(f"  Next GW: {next_gw} (to predict)")
print()

# Set target gameweeks
COMPLETED_GW = 16  # The gameweek that just finished (need to scrape this)
TARGET_GW = 17     # The gameweek we want to predict
print(f"Scraping completed GW: {COMPLETED_GW}")
print(f"Target for prediction: GW{TARGET_GW}")
print()

# ============================================================================
# STEP 2: SCRAPE GW16 DATA FROM FBREF
# ============================================================================
print("STEP 2: Scraping GW16 data from FBRef...")
print("=" * 80)

gw16_dir = DATA_DIR / SEASON / f"gw{COMPLETED_GW}"

# Check if we already have GW16 data
if gw16_dir.exists():
    existing_matches = [m for m in gw16_dir.iterdir() if m.is_dir()]
    print(f"  Found existing GW{COMPLETED_GW} data: {len(existing_matches)} matches")
    
    # Ask whether to re-scrape (in notebook context, we'll check match count)
    # FPL has 10 matches per gameweek typically
    if len(existing_matches) >= 10:
        print(f"  ✓ GW{COMPLETED_GW} data appears complete, skipping scrape")
        SKIP_SCRAPE = True
    else:
        print(f"  ⚠️ GW{COMPLETED_GW} data incomplete ({len(existing_matches)}/10 matches), will re-scrape")
        SKIP_SCRAPE = False
else:
    print(f"  GW{COMPLETED_GW} data not found, will scrape from FBRef")
    SKIP_SCRAPE = False

if not SKIP_SCRAPE:
    # Import and run the scraper
    from scrape_gameweek import scrape_gameweek
    
    print()
    scrape_result = scrape_gameweek(
        gameweek=COMPLETED_GW,
        season=SEASON,
        delete_existing=True,
        verbose=True
    )
    
    if scrape_result['matches_scraped'] == 0:
        print("\n⚠️ WARNING: No matches were scraped for GW{COMPLETED_GW}!")
        print("    This could mean:")
        print("    - GW16 matches haven't been played yet")
        print("    - FBRef hasn't updated match reports yet")
        print("    - There was a connection issue")
        print("\n    Predictions will use data up to the latest available GW.")
    else:
        print(f"\n✓ Successfully scraped {scrape_result['matches_scraped']} GW{COMPLETED_GW} matches")

print()

# ============================================================================
# STEP 3: Get GW17 fixtures from FPL API
# ============================================================================
print("STEP 3: Fetching GW17 fixtures from FPL API...")
fixtures_df = get_next_gw_fixtures(TARGET_GW)
print()

# Get teams playing in GW17
teams_playing = list(set(fixtures_df['home_team'].tolist() + fixtures_df['away_team'].tolist()))
print(f"Teams playing in GW{TARGET_GW}: {len(teams_playing)}")

# Save fixtures
fixtures_path = DATA_DIR / "upcoming" / f"gw{TARGET_GW}_fixtures.csv"
fixtures_path.parent.mkdir(parents=True, exist_ok=True)
fixtures_df.to_csv(fixtures_path, index=False)
print(f"Saved fixtures to: {fixtures_path}")
print()

# ============================================================================
# STEP 4: TUNE MODELS (with feature selection and Optuna)
# ============================================================================
print("STEP 4: Tuning models with feature selection and Optuna...")
print(f"  Using data from GW1-{COMPLETED_GW} to predict GW{TARGET_GW}")
print("=" * 80)

# Initialize pipeline
pipeline = FPLPredictionPipeline(data_dir=str(DATA_DIR))

# Configure tuning: RFE feature selection (10 features) + Optuna hyperparameter optimization
config_tune = TuneConfig(
    models=['goals', 'assists', 'minutes', 'defcon', 'bonus'],
    test_size=0.2,
    random_state=42,
    feature_selection='rfe',      # Recursive Feature Elimination
    feature_selection_k=10,       # Keep top 10 features
    optuna_trials=30,             # 30 Optuna trials for hyperparameter optimization
    optuna_timeout=300,           # 5 minute timeout per model
)

print("\nTuning models (this will take a few minutes)...")
print("  - Feature selection: RFE (top 10 features)")
print("  - Hyperparameter optimization: Optuna (30 trials)")
print()

# Tune models - this stores optimized models in pipeline.models
eval_results = pipeline.tune(config=config_tune, verbose=True)

# ============================================================================
# STEP 5: MODEL ACCURACY SUMMARY (from tuning results)
# ============================================================================
print()
print("=" * 80)
print("MODEL ACCURACY SUMMARY (from tuning)")
print("=" * 80)

print()
print(f"{'Model':<15} {'MAE':>10} {'RMSE':>10} {'R²':>10} {'Samples':>10}")
print("-" * 55)
for name, metrics in eval_results.items():
    print(f"{name.upper():<15} {metrics.mae:>10.4f} {metrics.rmse:>10.4f} {metrics.r2:>10.4f} {metrics.samples:>10,}")
print("-" * 55)

# ============================================================================
# STEP 5.5: RUN PREDICTIONS WITH TUNED MODELS
# ============================================================================
print()
print("=" * 80)
print("RUNNING PREDICTIONS WITH TUNED MODELS")
print("=" * 80)
print("Now using the tuned models (10 features, optimized hyperparameters) for predictions...")
print()

# Run predictions - this will use the existing tuned models in pipeline.models
predictions_df = pipeline.run(target_gw=TARGET_GW, target_season=SEASON, verbose=True)

# ============================================================================
# STEP 6: PLAYER PROJECTIONS TABLE
# ============================================================================
print()
print("=" * 80)
print(f"GW{TARGET_GW} PLAYER PROJECTIONS - TOP 50 BY EXPECTED POINTS")
print("=" * 80)

# Prepare display columns
display_cols = [
    'player_name', 'team', 'fpl_position', 'opponent', 'is_home',
    'pred_minutes', 'pred_exp_goals', 'pred_exp_assists', 
    'pred_cs_prob', 'pred_defcon_prob', 'pred_bonus', 'exp_total_pts'
]

# Filter to players with predicted minutes > 30 and get top 50
top_players = predictions_df[predictions_df['pred_minutes'] >= 30].copy()
top_players = top_players.nlargest(50, 'exp_total_pts')

# Rename columns for display
top_players_display = top_players[display_cols].copy()
top_players_display.columns = [
    'Player', 'Team', 'Pos', 'Opponent', 'Home',
    'Mins', 'xG', 'xA', 'CS%', 'Def%', 'Bonus', 'xPts'
]

# Format numeric columns
top_players_display['Mins'] = top_players_display['Mins'].round(0).astype(int)
top_players_display['xG'] = top_players_display['xG'].round(3)
top_players_display['xA'] = top_players_display['xA'].round(3)
top_players_display['CS%'] = (top_players_display['CS%'] * 100).round(1).astype(str) + '%'
top_players_display['Def%'] = (top_players_display['Def%'] * 100).round(1).astype(str) + '%'
top_players_display['Bonus'] = top_players_display['Bonus'].round(2)
top_players_display['xPts'] = top_players_display['xPts'].round(2)
top_players_display['Home'] = top_players_display['Home'].map({1: 'H', 0: 'A'})

# Reset index for clean display
top_players_display = top_players_display.reset_index(drop=True)
top_players_display.index = top_players_display.index + 1  # Start from 1

print()
print(top_players_display.to_string())

# ============================================================================
# STEP 7: POSITION BREAKDOWN
# ============================================================================
print()
print("=" * 80)
print("TOP PLAYERS BY POSITION")
print("=" * 80)

for pos in ['GK', 'DEF', 'MID', 'FWD']:
    pos_players = predictions_df[
        (predictions_df['fpl_position'] == pos) & 
        (predictions_df['pred_minutes'] >= 30)
    ].nlargest(5, 'exp_total_pts')
    
    print(f"\n{pos} - Top 5:")
    print("-" * 60)
    for _, row in pos_players.iterrows():
        home_away = "(H)" if row['is_home'] == 1 else "(A)"
        print(f"  {row['player_name']:25s} vs {row['opponent']:15s} {home_away} | xPts: {row['exp_total_pts']:.2f}")


# ============================================================================
# STEP 8: Save predictions
# ============================================================================
print()
print("=" * 80)
output_path = DATA_DIR / 'predictions' / f'gw{TARGET_GW}_{SEASON}_all_predictions.csv'
predictions_df.to_csv(output_path, index=False)
print(f"✓ Full predictions saved to: {output_path}")
print(f"✓ Total players predicted: {len(predictions_df)}")
print("=" * 80)


GW17 2025-26 PREDICTIONS USING TUNED MODELS
Timestamp: 2025-12-18 21:56:07

STEP 1: Checking FPL API for gameweek status...
Checking FPL API for current gameweek...
  Current GW: 16, Next GW: 17
  Current GW: 16 (just completed/in progress)
  Next GW: 17 (to predict)

Scraping completed GW: 16
Target for prediction: GW17

STEP 2: Scraping GW16 data from FBRef...
  Found existing GW16 data: 10 matches
  ✓ GW16 data appears complete, skipping scrape

STEP 3: Fetching GW17 fixtures from FPL API...

[1] Fetching GW17 fixtures from FPL API...
  Found 10 fixtures for GW17:
    Newcastle vs Chelsea
    Bournemouth vs Burnley
    Brighton vs Sunderland
    Man City vs West Ham
    Wolves vs Brentford
    Spurs vs Liverpool
    Everton vs Arsenal
    Leeds vs Crystal Palace
    Aston Villa vs Man Utd
    Fulham vs Nott'm Forest

Teams playing in GW17: 20
Saved fixtures to: c:\Users\dpfin\OneDrive\Desktop\ultimate_p_proj\data\upcoming\gw17_fixtures.csv

STEP 4: Tuning models with feature selecti

[I 2025-12-18 21:57:13,843] A new study created in memory with name: no-name-6417ec16-ac1d-4ae8-8f52-34a34a4d4502


  Computed 119 rolling features

[3] Splitting data for evaluation...
    Total samples (played 1+ min): 49,583

TUNING: GOALS MODEL
  Running Optuna optimization (30 trials)...


Best trial: 0. Best value: 0.274734:   3%|▎         | 1/30 [00:01<00:36,  1.27s/it, 1.27/300 seconds]

[I 2025-12-18 21:57:15,109] Trial 0 finished with value: 0.2747342223110047 and parameters: {'n_estimators': 250, 'max_depth': 10, 'learning_rate': 0.1205712628744377, 'min_child_weight': 6, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481, 'reg_alpha': 3.3323645788192616e-08, 'reg_lambda': 0.6245760287469893}. Best is trial 0 with value: 0.2747342223110047.


...

[I 2025-12-18 22:05:04,385] Trial 14 finished with value: 0.310096500072015 and parameters: {'n_estimators': 142, 'max_depth': 8, 'learning_rate': 0.05487145059380789, 'min_child_weight': 5}. Best is trial 13 with value: 0.307792020740314.


Best trial: 13. Best value: 0.307792:  53%|█████▎    | 16/30 [04:58<04:19, 18.50s/it, 298.72/300 seconds]

[I 2025-12-18 22:05:22,664] Trial 15 finished with value: 0.3149935186518796 and parameters: {'n_estimators': 103, 'max_depth': 7, 'learning_rate': 0.13652280192305127, 'min_child_weight': 8}. Best is trial 13 with value: 0.307792020740314.


Best trial: 13. Best value: 0.307792:  57%|█████▋    | 17/30 [05:17<04:02, 18.68s/it, 317.48/300 seconds]


[I 2025-12-18 22:05:41,417] Trial 16 finished with value: 0.3126890393201786 and parameters: {'n_estimators': 166, 'max_depth': 8, 'learning_rate': 0.07369477793265464, 'min_child_weight': 6}. Best is trial 13 with value: 0.307792020740314.
    Best MAE: 0.3078
    Best params: {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.1546519004407837, 'min_child_weight': 7, 'random_state': 42}
  Train samples: 27,770 (up to GW38)
  Test samples: 6,943 (from GW1)
  Feature selection: RFE (top 10 features)
    Selected 10 features
    Top 5: ['key_passes_per90_roll3', 'tackles_per90_roll5', 'interceptions_per90_roll5', 'recoveries_per90_roll5', 'sca_per90_roll3']
  Target: bonus (0-3)
  Actual mean: 0.1337
  Predicted mean: 0.2001
  Training final model on all 34,713 samples...

  BONUS MODEL METRICS
  Samples: 6,943
  MAE:     0.3041
  RMSE:    0.6218
  R²:      -0.3132
  Selected Features: 10
  Best Params (Optuna):
    n_estimators: 150
    max_depth: 8
    learning_rate: 0.1546519004

In [4]:
goals_model = pipeline.get_model('goals')
print(f"Goals Model Features ({len(goals_model.FEATURES)}):")
print("-" * 60)
for i, feat in enumerate(goals_model.FEATURES, 1):
    print(f"  {i:2}. {feat}")

goals_model = pipeline.get_model('assists')
print(f"assists Model Features ({len(goals_model.FEATURES)}):")
print("-" * 60)
for i, feat in enumerate(goals_model.FEATURES, 1):
    print(f"  {i:2}. {feat}")
goals_model = pipeline.get_model('defcon')
print(f"defcon Model Features ({len(goals_model.FEATURES)}):")
print("-" * 60)
for i, feat in enumerate(goals_model.FEATURES, 1):
    print(f"  {i:2}. {feat}")
goals_model = pipeline.get_model('minutes')
print(f"minutes Model Features ({len(goals_model.FEATURES)}):")
print("-" * 60)
for i, feat in enumerate(goals_model.FEATURES, 1):
    print(f"  {i:2}. {feat}")
goals_model = pipeline.get_model('bonus')
print(f"bonus Model Features ({len(goals_model.FEATURES)}):")
print("-" * 60)
for i, feat in enumerate(goals_model.FEATURES, 1):
    print(f"  {i:2}. {feat}")

Goals Model Features (10):
------------------------------------------------------------
   1. xg_per90_roll1
   2. shots_per90_roll1
   3. shots_per90_roll3
   4. key_passes_per90_roll3
   5. goals_per90_roll5
   6. goals_per90_roll10
   7. shots_per90_roll10
   8. xg_per90_roll10
   9. shots_per90_roll5
  10. team_goals_roll5
assists Model Features (10):
------------------------------------------------------------
   1. goals_per90_roll1
   2. goals_per90_roll3
   3. xg_per90_roll1
   4. shots_per90_roll1
   5. sca_per90_roll3
   6. sca_per90_roll5
   7. team_goals_roll5
   8. opp_conceded_roll5
   9. opp_shots_roll10
  10. opp_shots_roll20
defcon Model Features (10):
------------------------------------------------------------
   1. defcon_per90_roll5
   2. defcon_per90_roll10
   3. defcon_per90_last1
   4. interceptions_per90_roll5
   5. blocks_per90_roll5
   6. recoveries_per90_roll5
   7. hit_threshold_roll5
   8. is_defender
   9. is_midfielder
  10. opp_xg_roll5
minutes Model Fe

In [5]:
bootstrap = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()

# Map element_type to position string
POS_MAP = {1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD'}

# Build FPL player lookup by name variants
fpl_players = {}
for p in bootstrap['elements']:
    pos = POS_MAP.get(p['element_type'], 'MID')
    fpl_id = p['id']
    # Store multiple name variants
    names = [
        p['web_name'],  # e.g., "Salah"
        f"{p['first_name']} {p['second_name']}",  # e.g., "Mohamed Salah"
        p['second_name'],  # e.g., "Salah"
    ]
    for name in names:
        fpl_players[name.lower()] = {'fpl_id': fpl_id, 'fpl_position': pos, 'web_name': p['web_name']}
fpl_players

{'raya': {'fpl_id': 1, 'fpl_position': 'GK', 'web_name': 'Raya'},
 'david raya martín': {'fpl_id': 1, 'fpl_position': 'GK', 'web_name': 'Raya'},
 'raya martín': {'fpl_id': 1, 'fpl_position': 'GK', 'web_name': 'Raya'},
 'arrizabalaga': {'fpl_id': 2,
  'fpl_position': 'GK',
  'web_name': 'Arrizabalaga'},
 'kepa arrizabalaga revuelta': {'fpl_id': 2,
  'fpl_position': 'GK',
  'web_name': 'Arrizabalaga'},
 'arrizabalaga revuelta': {'fpl_id': 2,
  'fpl_position': 'GK',
  'web_name': 'Arrizabalaga'},
 'hein': {'fpl_id': 3, 'fpl_position': 'GK', 'web_name': 'Hein'},
 'karl hein': {'fpl_id': 3, 'fpl_position': 'GK', 'web_name': 'Hein'},
 'setford': {'fpl_id': 4, 'fpl_position': 'GK', 'web_name': 'Setford'},
 'tommy setford': {'fpl_id': 4, 'fpl_position': 'GK', 'web_name': 'Setford'},
 'gabriel': {'fpl_id': 5, 'fpl_position': 'DEF', 'web_name': 'Gabriel'},
 'gabriel dos santos magalhães': {'fpl_id': 5,
  'fpl_position': 'DEF',
  'web_name': 'Gabriel'},
 'dos santos magalhães': {'fpl_id': 5,
  'f

In [6]:
import pandas as pd
data = pd.DataFrame(fpl_players).T

In [7]:
test = "Raúl Jiménez"

In [8]:
test.split()[-1].lower()

'jiménez'

In [9]:
data.loc[data['web_name']==test.split()[-1].lower()]

,fpl_id,fpl_position,web_name


In [10]:
data.loc[data['web_name'].str.contains('Fernandes')]

,fpl_id,fpl_position,web_name
b.fernandes,449,MID,B.Fernandes
bruno borges fernandes,449,MID,B.Fernandes
borges fernandes,449,MID,B.Fernandes
m.fernandes,708,MID,M.Fernandes
mateus gonçalo espanha fernandes,708,MID,M.Fernandes
espanha fernandes,708,MID,M.Fernandes
